In [1]:
import findspark
findspark.init()
import pyspark

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('Assignment_4_and_5').getOrCreate()

In [3]:
#READING DATASET 
dataorg=(spark.read.option('header','true').csv('MoviesA3.csv', sep=";" , inferSchema=True)).persist()
data = dataorg

In [4]:
data.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Length: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Actor: string (nullable = true)
 |-- Actress: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Popularity: integer (nullable = true)
 |-- Awards: string (nullable = true)
 |-- Image: string (nullable = true)



In [5]:
data.columns

['Year',
 'Length',
 'Title',
 'Genre',
 'Actor',
 'Actress',
 'Director',
 'Popularity',
 'Awards',
 'Image']

#  <font color='green'>Assignment 4 and 5 - Spark Data Frames and Clustering</font>

### Type of Each Attribute

In [6]:
data.dtypes

[('Year', 'int'),
 ('Length', 'int'),
 ('Title', 'string'),
 ('Genre', 'string'),
 ('Actor', 'string'),
 ('Actress', 'string'),
 ('Director', 'string'),
 ('Popularity', 'int'),
 ('Awards', 'string'),
 ('Image', 'string')]

### Nominal data include country, gender, race, hair color etc. of a group of people.
### Ordinal data include having a position in class as “First” or “Second”.
### An interval scale is one where there is order and the difference between two values is meaningful. Examples of interval variables include: temperature (Farenheit), temperature (Celcius), pH, SAT score (200-800), credit score (300-850).
### A ratio variable, has all the properties of an interval variable, and also has a clear definition of 0.0. When the variable equals 0.0, there is none of that variable. Examples of ratio variables include: enzyme activity, dose amount, reaction rate, flow rate, concentration, pulse, weight, length, temperature in Kelvin (0.0 Kelvin really does mean “no heat”), survival time.

| Attribute | Type | Reason |
| ----------|------|--------------------------------------------------------------------------------------------------------- |
| Year | Interval  | Since there is a difference between two values is meaningful and hence this attribute lies in the Interval Type and can never be 0 |
| Length | Ordinal | Since it is a measurable attribute and hence this attribute lies in the Ordinal Type |
| Title | Nominal  | Since the attribute is not ordered and it lies in the category of a group of something and hence this attribute lies in the Nominal Type |
| Genre | Nominal  | Since the attribute is not ordered and it lies in the category of a group of something and hence this attribute lies in the Nominal Type |
| Actor | Nominal  | Since the attribute is not ordered and it lies in the category of a group of something and hence this attribute lies in the Nominal Type |
| Actress | Nominal  | Since the attribute is not ordered and it lies in the category of a group of something and hence this attribute lies in the Nominal Type |
| Director | Nominal  | Since the attribute is not ordered and it lies in the category of a group of something and hence this attribute lies in the Nominal Type |
| Popularity | Ratio  | Since the attribute has all the properties of an interval variable, and also has a clear definition of 0.0 and hence this attribute lies in the Ratio Type |
| Awards | Nominal  | Since the attribute is not ordered and it lies in the category of a group of something and hence this attribute lies in the Nominal Type |
Image | Nominal  | Since the attribute is not ordered and it lies in the category of a group of something and hence this attribute lies in the Nominal Type |

### Pre-Processing

In [7]:
data.filter(data.Year.isNull()).show(5)
data.filter(data.Year.isNull()).count()

+----+------+-----+-----+-----+-------+--------+----------+------+-----+
|Year|Length|Title|Genre|Actor|Actress|Director|Popularity|Awards|Image|
+----+------+-----+-----+-----+-------+--------+----------+------+-----+
+----+------+-----+-----+-----+-------+--------+----------+------+-----+



0

In [8]:
data.filter(data.Length.isNull()).show(5)
data.filter(data.Length.isNull()).count()

+----+------+--------------------+-------+---------------+--------------------+--------------------+----------+------+------------------+
|Year|Length|               Title|  Genre|          Actor|             Actress|            Director|Popularity|Awards|             Image|
+----+------+--------------------+-------+---------------+--------------------+--------------------+----------+------+------------------+
|1980|  null|Happy Birthday to Me| Horror|    Ford, Glenn|Anderson, Melissa...|    Thompson, J. Lee|        88|    No|   glennFord.png,,|
|1962|  null|              Dr. No| Action|  Connery, Sean|     Andress, Ursula|      Young, Terence|         7|    No| seanConnery.png,,|
|1989|  null|             Killjoy|Mystery|   Culp, Robert|       Basinger, Kim|Moxey, John Llewe...|        71|    No|NicholasCage.png,,|
|1982|  null|Catch a Rising St...| Comedy|Belzer, Richard|        Benatar, Pat|                null|        18|    No|NicholasCage.png,,|
|1989|  null|         Good Fellas|

67

In [9]:
data.filter(data.Genre.isNull()).show(5)
data.filter(data.Genre.isNull()).count()

+----+------+---------------+-----+-----------------+----------------+----------------+----------+------+------------------+
|Year|Length|          Title|Genre|            Actor|         Actress|        Director|Popularity|Awards|             Image|
+----+------+---------------+-----+-----------------+----------------+----------------+----------+------+------------------+
|1953|    61|White Lightning| null|Clements, Stanley|Blondell, Gloria|  Bernds, Edward|      null|    No|NicholasCage.png,,|
|1980|   180|     Wild Times| null|     Elliott, Sam|   Peyser, Penny|Compton, Richard|        75|    No|NicholasCage.png,,|
+----+------+---------------+-----+-----------------+----------------+----------------+----------+------+------------------+



2

In [10]:
data.filter(data.Actor.isNull()).show(5)
data.filter(data.Actor.isNull()).count()

+----+------+--------------------+-----+-----+-------+----------------+----------+------+--------------------+
|Year|Length|               Title|Genre|Actor|Actress|        Director|Popularity|Awards|               Image|
+----+------+--------------------+-----+-----+-------+----------------+----------+------+--------------------+
|1988|   100|Ciao Italia, Mado...|Music| null|Madonna|De Winter, Harry|        74|    No| NicholasCage.png,,,|
|1991|   118|Madonna, Truth or...|Music| null|Madonna|Keshishian, Alek|        54|    No| NicholasCage.png,,,|
|1990|    60|Immaculate Collec...|Music| null|Madonna|            null|        32|    No|NicholasCage.png,,,,|
|1987|    50|Madonna Live, The...|Music| null|Madonna|            null|        75|    No|NicholasCage.png,,,,|
|1990|     5|Madonna, Justify ...|Music| null|Madonna|            null|        77|    No|NicholasCage.png,,,,|
+----+------+--------------------+-----+-----+-------+----------------+----------+------+--------------------+
o

8

In [11]:
data.filter(data.Actress.isNull()).show(5)
data.filter(data.Actress.isNull()).count()

+----+------+--------------------+-------+---------------+-------+---------------+----------+------+--------------------+
|Year|Length|               Title|  Genre|          Actor|Actress|       Director|Popularity|Awards|               Image|
+----+------+--------------------+-------+---------------+-------+---------------+----------+------+--------------------+
|1953|    94|           Vera Cruz| Action|   Cooper, Gary|   null|Aldrich, Robert|        71|    No| NicholasCage.png,,,|
|1954|    91|              Apache|Western|Lancaster, Burt|   null|Aldrich, Robert|        78|    No|burtLancaster.png,,,|
|1977|   146|Twilight's Last G...|  Drama|Lancaster, Burt|   null|Aldrich, Robert|        84|    No|burtLancaster.png,,,|
|1979|   119|     Frisco Kid, The| Comedy|   Wilder, Gene|   null|Aldrich, Robert|        10|    No|  NicholasCage.png,,|
|1954|    30|   Bank on the Stars|  Drama|     Paar, Jack|   null|   Allen, Craig|      null|    No| NicholasCage.png,,,|
+----+------+-----------

378

In [12]:
data.filter(data.Director.isNull()).show(5)
data.filter(data.Director.isNull()).count()

+----+------+--------------------+------+---------------+---------------+--------+----------+------+-------------------+
|Year|Length|               Title| Genre|          Actor|        Actress|Director|Popularity|Awards|              Image|
+----+------+--------------------+------+---------------+---------------+--------+----------+------+-------------------+
|1992|    90|           Germicide| Drama|    Taylor, Rod|Andersson, Bibi|    null|        36|    No|NicholasCage.png,,,|
|1954|   103|       Elephant Walk| Drama|   Finch, Peter|  Andrews, Dana|    null|        11|    No|NicholasCage.png,,,|
|1985|    55|Gonzo Presents Mu...|Comedy|   Cleese, John| Andrews, Julie|    null|        88|    No|NicholasCage.png,,,|
|1991|   115|              Bataan|   War| Taylor, Robert|    Arnaz, Desi|    null|        68|    No|NicholasCage.png,,,|
|1991|    90|Henry, Portrait o...|Horror|Rooker, Michael|  Arnold, Tracy|    null|        69|    No| NicholasCage.png,,|
+----+------+-------------------

253

In [13]:
data.filter(data.Popularity.isNull()).show(5)
data.filter(data.Popularity.isNull()).count()

+----+------+------------------+-------+--------------------+----------------+-------------------+----------+------+-------------------+
|Year|Length|             Title|  Genre|               Actor|         Actress|           Director|Popularity|Awards|              Image|
+----+------+------------------+-------+--------------------+----------------+-------------------+----------+------+-------------------+
|1953|    61|   White Lightning|   null|   Clements, Stanley|Blondell, Gloria|     Bernds, Edward|      null|    No| NicholasCage.png,,|
|1986|    90|Knights & Emeralds|  Drama|    Leadbitter, Bill|  Hills, Beverly|          Emes, Ian|      null|    No| NicholasCage.png,,|
|1927|    62|    Drop Kick, The|  Drama|Barthelmess, Richard|   Kent, Barbara|      Webb, Millard|      null|    No|  NicholasCage.png,|
|1923|    57|      Desert Rider|Western|         Hoxie, Jack|  Nelson, Evelyn|Bradbury, Robert N.|      null|    No| NicholasCage.png,,|
|1954|    30| Bank on the Stars|  Drama| 

6

In [14]:
data.filter(data.Awards.isNull()).show(5)
data.filter(data.Awards.isNull()).count()

+----+------+-----+-----+-----+-------+--------+----------+------+-----+
|Year|Length|Title|Genre|Actor|Actress|Director|Popularity|Awards|Image|
+----+------+-----+-----+-----+-------+--------+----------+------+-----+
+----+------+-----+-----+-----+-------+--------+----------+------+-----+



0

In [15]:
data.filter(data.Image.isNull()).show(5)
data.filter(data.Image.isNull()).count()

+----+------+-----+-----+-----+-------+--------+----------+------+-----+
|Year|Length|Title|Genre|Actor|Actress|Director|Popularity|Awards|Image|
+----+------+-----+-----+-----+-------+--------+----------+------+-----+
+----+------+-----+-----+-----+-------+--------+----------+------+-----+



0

## After checking for NULL values in the dataframe we have found out that "Length" has 67 null values, "Genre" has 2 null values, "Actor" has 8 null values, "Actress" has 378 Null values, "Director" has 253 Null values and "Popularity" has 6 null values, the rest has no null values.
## Now I will impute those null values using different techniques

1. Length 
#### Since the movies with the same genre has almost the same length on average, so I have decided to take the mean of length of each Genre and impute the values for the Length Attribute for the Genre it has

In [16]:
# First step is to check the Genre we need to calculate the mean of the movies length for
data.select('Genre').filter(data.Length.isNull()).distinct().show()

+--------+
|   Genre|
+--------+
|   Drama|
|     War|
| Mystery|
|   Music|
|  Horror|
| Western|
|  Comedy|
|  Action|
|Westerns|
+--------+



In [17]:
data.filter(data.Length.isNull()).count()

67

In [18]:
from pyspark.sql.functions import mean as _mean, col

drama = data.select(data.Length).filter(data.Genre == 'Drama')

df_stats = drama.select(
    _mean(col('Length')).alias('mean')
).collect()

mean_drama = df_stats[0]['mean']
mean_drama

113.30455259026688

In [19]:
war = data.select(data.Length).filter(data.Genre == 'War')

df_stats = war.select(
    _mean(col('Length')).alias('mean')
).collect()

mean_war = df_stats[0]['mean']
mean_war

116.90625

In [20]:
mystery = data.select(data.Length).filter(data.Genre == 'Mystery')

df_stats = mystery.select(
    _mean(col('Length')).alias('mean')
).collect()

mean_mystery = df_stats[0]['mean']
mean_mystery

103.00990099009901

In [21]:
music = data.select(data.Length).filter(data.Genre == 'Music')

df_stats = music.select(
    _mean(col('Length')).alias('mean')
).collect()

mean_music = df_stats[0]['mean']
mean_music

100.48780487804878

In [22]:
horror = data.select(data.Length).filter(data.Genre == 'Horror')

df_stats = horror.select(
    _mean(col('Length')).alias('mean')
).collect()

mean_horror = df_stats[0]['mean']
mean_horror

93.92727272727272

In [23]:
western = data.select(data.Length).filter(data.Genre == 'Western')

df_stats = western.select(
    _mean(col('Length')).alias('mean')
).collect()

mean_western = df_stats[0]['mean']
mean_western

93.0091743119266

In [24]:
comedy = data.select(data.Length).filter(data.Genre == 'Comedy')

df_stats = comedy.select(
    _mean(col('Length')).alias('mean')
).collect()

mean_comedy = df_stats[0]['mean']
mean_comedy

96.50540540540541

In [25]:
action = data.select(data.Length).filter(data.Genre == 'Action')

df_stats = action.select(
    _mean(col('Length')).alias('mean')
).collect()

mean_action = df_stats[0]['mean']
mean_action

104.5

In [26]:
westerns = data.select(data.Length).filter(data.Genre == 'Westerns')

df_stats = westerns.select(
    _mean(col('Length')).alias('mean')
).collect()

mean_westerns = df_stats[0]['mean']
mean_westerns

124.8

In [27]:
dt8 = data.filter(data.Length.isNotNull())
dt0 = data.filter((data.Length.isNull())&(data.Genre == 'Drama')).na.fill(value = mean_drama,subset=["Length"])
dt0.show(5)

+----+------+--------------------+-----+---------------+----------------+----------------+----------+------+-------------------+
|Year|Length|               Title|Genre|          Actor|         Actress|        Director|Popularity|Awards|              Image|
+----+------+--------------------+-----+---------------+----------------+----------------+----------+------+-------------------+
|1989|   113|         Good Fellas|Drama|De Niro, Robert|Bracco, Lorraine|Scorsese, Martin|        15|    No| NicholasCage.png,,|
|1943|   113|     A Guy Named Joe|Drama| Tracy, Spencer|    Dunne, Irene| Fleming, Victor|        42|    No| spencerTracy.png,,|
|1991|   113|         Coming Home|Drama|    Voight, Jon|     Fonda, Jane|            null|         1|   Yes|NicholasCage.png,,,|
|1988|   113|        Five Corners|Drama|   Robbins, Tim|   Foster, Jodie|            null|        88|    No|NicholasCage.png,,,|
|1955|   113|Blackboard Jungle...|Drama|    Ford, Glenn|   Francis, Anne| Brooks, Richard|       

In [28]:
dt1 = data.filter((data.Length.isNull())&(data.Genre ==  'War')).na.fill(value = mean_war,subset=["Length"])
dt1.show(5)

+----+------+--------------------+-----+-----------+-------+--------+----------+------+-----------------+
|Year|Length|               Title|Genre|      Actor|Actress|Director|Popularity|Awards|            Image|
+----+------+--------------------+-----+-----------+-------+--------+----------+------+-----------------+
|1991|   116|John Wayne Collec...|  War|Wayne, John|   null|    null|        49|    No|  johnWayne.png,,|
|1992|   116|John Wayne Collec...|  War|Wayne, John|   null|    null|         3|    No|johnWayne.png,,,,|
+----+------+--------------------+-----+-----------+-------+--------+----------+------+-----------------+



In [29]:
dt2 = data.filter((data.Length.isNull())&(data.Genre == 'Mystery')).na.fill(value = mean_mystery,subset=["Length"])
dt2.show(5)

+----+------+------------------+-------+-------------+---------------+--------------------+----------+------+-------------------+
|Year|Length|             Title|  Genre|        Actor|        Actress|            Director|Popularity|Awards|              Image|
+----+------+------------------+-------+-------------+---------------+--------------------+----------+------+-------------------+
|1989|   103|           Killjoy|Mystery| Culp, Robert|  Basinger, Kim|Moxey, John Llewe...|        71|    No| NicholasCage.png,,|
|1988|   103|  White of the Eye|Mystery| Keith, David|Moriarty, Cathy|     Cammell, Donald|        48|    No| NicholasCage.png,,|
|1988|   103|Still of the Night|Mystery|Scheider, Roy|  Streep, Meryl|      Benton, Robert|        42|    No|  merylStreep.png,,|
|1992|   103| Mummy's Hand, The|Mystery|  Foran, Dick|           null|                null|        54|    No|NicholasCage.png,,,|
|1989|   103|        Saint, The|Mystery| Moore, Roger|           null|                null

In [30]:
dt3 = data.filter((data.Length.isNull())&(data.Genre == 'Music')).na.fill(value = mean_music,subset=["Length"])
dt3.show(5)

+----+------+--------------------+-----+----------+-------------+--------+----------+------+-------------------+
|Year|Length|               Title|Genre|     Actor|      Actress|Director|Popularity|Awards|              Image|
+----+------+--------------------+-----+----------+-------------+--------+----------+------+-------------------+
|1984|   100|Gary Numan - Berz...|Music|Webb, John|Taylor, Karen|    null|        60|    No|NicholasCage.png,,,|
|1990|   100|Sounds of the Sev...|Music|Jones, Tom|Welch, Raquel|    null|        13|    No|NicholasCage.png,,,|
+----+------+--------------------+-----+----------+-------------+--------+----------+------+-------------------+



In [31]:
dt4 = data.filter((data.Length.isNull())&(data.Genre == 'Horror')).na.fill(value = mean_horror,subset=["Length"])
dt4.show(5)

+----+------+--------------------+------+--------------+--------------------+----------------+----------+------+-------------------+
|Year|Length|               Title| Genre|         Actor|             Actress|        Director|Popularity|Awards|              Image|
+----+------+--------------------+------+--------------+--------------------+----------------+----------+------+-------------------+
|1980|    93|Happy Birthday to Me|Horror|   Ford, Glenn|Anderson, Melissa...|Thompson, J. Lee|        88|    No|    glennFord.png,,|
|1988|    93|  Mama's Dirty Girls|Horror|Currie, Sondra|     Grahame, Gloria|            null|        62|    No|NicholasCage.png,,,|
+----+------+--------------------+------+--------------+--------------------+----------------+----------+------+-------------------+



In [32]:
dt5 = data.filter((data.Length.isNull())&(data.Genre == 'Western')).na.fill(value = mean_western,subset=["Length"])
dt5.show(5)

+----+------+--------------------+-------+--------------+--------------+--------------+----------+------+-------------------+
|Year|Length|               Title|  Genre|         Actor|       Actress|      Director|Popularity|Awards|              Image|
+----+------+--------------------+-------+--------------+--------------+--------------+----------+------+-------------------+
|1931|    93|     Range Feud, The|Western|   Wayne, John|Fleming, Susan|Lederman, Ross|        51|    No|     johnWayne.png,|
|1989|    93|Death Valley Days...|Western|   Caan, James|          null|          null|         9|    No|NicholasCage.png,,,|
|1989|    93|Death Valley Days...|Western|Reagan, Ronald|          null|          null|         1|    No|NicholasCage.png,,,|
|1993|    93|Duke, The Films o...|Western|   Wayne, John|          null|          null|        70|    No|   johnWayne.png,,,|
|1932|    93|   Hurricane Express|Western|   Wayne, John|          null|          null|         7|    No|  johnWayne.p

In [33]:
dt6 = data.filter((data.Length.isNull())&(data.Genre ==  'Comedy')).na.fill(value = mean_comedy,subset=["Length"])
dt6.show(5)

+----+------+--------------------+------+--------------------+---------------+-----------------+----------+------+-------------------+
|Year|Length|               Title| Genre|               Actor|        Actress|         Director|Popularity|Awards|              Image|
+----+------+--------------------+------+--------------------+---------------+-----------------+----------+------+-------------------+
|1982|    96|Catch a Rising St...|Comedy|     Belzer, Richard|   Benatar, Pat|             null|        18|    No| NicholasCage.png,,|
|1987|    96|           Mannequin|Comedy|    McCarthy, Andrew|  Cattrall, Kim|Gottlieb, Michael|        23|    No| NicholasCage.png,,|
|1990|    96|          Party, The|Comedy|      Sellers, Peter|Champion, Marge|   Edwards, Blake|        32|    No|  NicholasCage.png,|
|1991|    96|             Why Me?|Comedy|Lambert, Christopher|    Greist, Kim|             null|        74|    No|NicholasCage.png,,,|
|1990|    96|        Crazy People|Comedy|       Moore, 

In [34]:
dt7 = data.filter((data.Length.isNull())&(data.Genre == 'Westerns')).na.fill(value = mean_westerns,subset=["Length"])
dt7.show(5)
dt9 = data.filter((data.Length.isNull())&(data.Genre == 'Action')).na.fill(value = mean_action,subset=["Length"])
dt9.show(5)

+----+------+--------------------+--------+---------------+-------+--------+----------+------+--------------------+
|Year|Length|               Title|   Genre|          Actor|Actress|Director|Popularity|Awards|               Image|
+----+------+--------------------+--------+---------------+-------+--------+----------+------+--------------------+
|1991|   124|Clint Eastwood Co...|Westerns|Eastwood, Clint|   null|    null|        11|    No|clintEastwood.png,,,|
+----+------+--------------------+--------+---------------+-------+--------+----------+------+--------------------+

+----+------+--------------------+------+-----------------+-------------------+--------------------+----------+------+-------------------+
|Year|Length|               Title| Genre|            Actor|            Actress|            Director|Popularity|Awards|              Image|
+----+------+--------------------+------+-----------------+-------------------+--------------------+----------+------+-------------------+
|1

In [35]:
from functools import reduce
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)
data2 = unionAll(dt8,dt0,dt1,dt2,dt3,dt4,dt5,dt6,dt7,dt9)
data2.show(5)
print((data2.count(), len(data2.columns)))

+----+------+--------------------+------+-------------------+---------------+-----------------+----------+------+------------------+
|Year|Length|               Title| Genre|              Actor|        Actress|         Director|Popularity|Awards|             Image|
+----+------+--------------------+------+-------------------+---------------+-----------------+----------+------+------------------+
|1990|   111|Tie Me Up! Tie Me...|Comedy|  Banderas, Antonio|Abril, Victoria| Almod�var, Pedro|        68|    No|NicholasCage.png,,|
|1991|   113|          High Heels|Comedy|       Bos�, Miguel|Abril, Victoria| Almod�var, Pedro|        68|    No|NicholasCage.png,,|
|1983|   104|      Dead Zone, The|Horror|Walken, Christopher|  Adams, Brooke|Cronenberg, David|        79|    No| NicholasCage.png,|
|1979|   122|                Cuba|Action|      Connery, Sean|  Adams, Brooke|  Lester, Richard|         6|    No| seanConnery.png,,|
|1978|    94|      Days of Heaven| Drama|      Gere, Richard|  Adams,

2. Genre 
#### Genre has 2 missing values here I will check the Director and mode of the movies he/she directs and put it there

In [36]:
# First step is to check the Director we need to calculate the mode of the movies Genre for
data.select('Director').filter(data.Genre.isNull()).distinct().show()

+----------------+
|        Director|
+----------------+
|Compton, Richard|
|  Bernds, Edward|
+----------------+



In [37]:
from pyspark.sql.functions import col

cnts = data.filter(data.Director == 'Compton, Richard').groupBy("Genre").count()
cnts.show()

data.filter(data.Director == 'Compton, Richard').show()
cnts = data.filter(data.Director == 'Bernds, Edward').groupBy("Genre").count()
cnts.show()
data.filter(data.Director == 'Bernds, Edward').show()

# Since we only have one value here we can't use Director hence I have decided to drop these two values 
# as only two values out of 1659 would not affect the dataset 

+-----+-----+
|Genre|count|
+-----+-----+
| null|    1|
+-----+-----+

+----+------+----------+-----+------------+-------------+----------------+----------+------+------------------+
|Year|Length|     Title|Genre|       Actor|      Actress|        Director|Popularity|Awards|             Image|
+----+------+----------+-----+------------+-------------+----------------+----------+------+------------------+
|1980|   180|Wild Times| null|Elliott, Sam|Peyser, Penny|Compton, Richard|        75|    No|NicholasCage.png,,|
+----+------+----------+-----+------------+-------------+----------------+----------+------+------------------+

+-----+-----+
|Genre|count|
+-----+-----+
| null|    1|
+-----+-----+

+----+------+---------------+-----+-----------------+----------------+--------------+----------+------+------------------+
|Year|Length|          Title|Genre|            Actor|         Actress|      Director|Popularity|Awards|             Image|
+----+------+---------------+-----+----------------

In [38]:
# Dropping Null Values
print(data2.na.drop(subset=["Genre"]).count())
data2 = data2.na.drop(subset=["Genre"])

1657


3. Actor
#### Actor has 8 missing values here I will check the Genre where the Actor is same and for other values and put it in there

In [39]:
# First step is to check the Director we need to calculate the mode of the movies Genre for
data.filter(data.Actor.isNull()).show(5)
# Since we have Null values for the Actors working in Music Genre so I am trying to get the mode of the Actor who worked in Music Genre
data.select('Actor').filter(data.Genre == 'Music').distinct().show(5)
music_actors=data.select('*').filter(data.Genre == 'Music').groupBy('Actor').count().sort((col('count')).desc())
music_actors.show(5)

+----+------+--------------------+-----+-----+-------+----------------+----------+------+--------------------+
|Year|Length|               Title|Genre|Actor|Actress|        Director|Popularity|Awards|               Image|
+----+------+--------------------+-----+-----+-------+----------------+----------+------+--------------------+
|1988|   100|Ciao Italia, Mado...|Music| null|Madonna|De Winter, Harry|        74|    No| NicholasCage.png,,,|
|1991|   118|Madonna, Truth or...|Music| null|Madonna|Keshishian, Alek|        54|    No| NicholasCage.png,,,|
|1990|    60|Immaculate Collec...|Music| null|Madonna|            null|        32|    No|NicholasCage.png,,,,|
|1987|    50|Madonna Live, The...|Music| null|Madonna|            null|        75|    No|NicholasCage.png,,,,|
|1990|     5|Madonna, Justify ...|Music| null|Madonna|            null|        77|    No|NicholasCage.png,,,,|
+----+------+--------------------+-----+-----+-------+----------------+----------+------+--------------------+
o

In [40]:
# Taking mode of Actor with most movies and adding it to the null values
actors=data.groupBy('Actor').count().sort((col('count')).desc())
actors.show(5)

maximum_val=actors.agg({"count": "max"}).first()[0]

actor_with_most_movies=actors.select('Actor').filter((col('count')==maximum_val)).collect()[0][0] # Mode

+---------------+-----+
|          Actor|count|
+---------------+-----+
|    Wayne, John|   81|
|Eastwood, Clint|   31|
|  Connery, Sean|   30|
|   Newman, Paul|   27|
| Brando, Marlon|   24|
+---------------+-----+
only showing top 5 rows



In [41]:
print(actor_with_most_movies)
data2 = data2.na.fill(value = actor_with_most_movies,subset=["Actor"])
data2.show(5)
print((data2.count(), len(data2.columns)))

Wayne, John
+----+------+--------------------+------+-------------------+---------------+-----------------+----------+------+------------------+
|Year|Length|               Title| Genre|              Actor|        Actress|         Director|Popularity|Awards|             Image|
+----+------+--------------------+------+-------------------+---------------+-----------------+----------+------+------------------+
|1990|   111|Tie Me Up! Tie Me...|Comedy|  Banderas, Antonio|Abril, Victoria| Almod�var, Pedro|        68|    No|NicholasCage.png,,|
|1991|   113|          High Heels|Comedy|       Bos�, Miguel|Abril, Victoria| Almod�var, Pedro|        68|    No|NicholasCage.png,,|
|1983|   104|      Dead Zone, The|Horror|Walken, Christopher|  Adams, Brooke|Cronenberg, David|        79|    No| NicholasCage.png,|
|1979|   122|                Cuba|Action|      Connery, Sean|  Adams, Brooke|  Lester, Richard|         6|    No| seanConnery.png,,|
|1978|    94|      Days of Heaven| Drama|      Gere, Rich

4. Actress
#### Actress has 378 missing values here I will check with the same technique as used for Actor and then see what can we do to impute the values first

In [42]:
# First step is to check the Director we need to calculate the mode of the movies Genre for
data2.filter(data2.Actress.isNull()).show(5)
# Since we have Null values for the Actors working in Music Genre so I am trying to get the mode of the Actor who worked in Music Genre
data2.select('Genre').filter(data2.Actress.isNull()).distinct().show()

+----+------+--------------------+-------+---------------+-------+---------------+----------+------+--------------------+
|Year|Length|               Title|  Genre|          Actor|Actress|       Director|Popularity|Awards|               Image|
+----+------+--------------------+-------+---------------+-------+---------------+----------+------+--------------------+
|1953|    94|           Vera Cruz| Action|   Cooper, Gary|   null|Aldrich, Robert|        71|    No| NicholasCage.png,,,|
|1954|    91|              Apache|Western|Lancaster, Burt|   null|Aldrich, Robert|        78|    No|burtLancaster.png,,,|
|1977|   146|Twilight's Last G...|  Drama|Lancaster, Burt|   null|Aldrich, Robert|        84|    No|burtLancaster.png,,,|
|1979|   119|     Frisco Kid, The| Comedy|   Wilder, Gene|   null|Aldrich, Robert|        10|    No|  NicholasCage.png,,|
|1954|    30|   Bank on the Stars|  Drama|     Paar, Jack|   null|   Allen, Craig|      null|    No| NicholasCage.png,,,|
+----+------+-----------

In [43]:
drama_actresses=data2.select('*').filter((data2.Genre == 'Drama') & (data2.Actress.isNotNull())).groupBy('Actress').count().sort((col('count')).desc())
war_actresses=data2.select('*').filter((data2.Genre == 'War') & (data2.Actress.isNotNull())).groupBy('Actress').count().sort((col('count')).desc())
mystery_actresses=data2.select('*').filter((data2.Genre == 'Mystery') & (data2.Actress.isNotNull())).groupBy('Actress').count().sort((col('count')).desc())
music_actresses=data2.select('*').filter((data2.Genre == 'Music') & (data2.Actress.isNotNull())).groupBy('Actress').count().sort((col('count')).desc())
sci_fi_actresses=data2.select('*').filter((data2.Genre == 'Science Fiction') & (data2.Actress.isNotNull())).groupBy('Actress').count().sort((col('count')).desc())
horror_actresses=data2.select('*').filter((data2.Genre == 'Horror') & (data2.Actress.isNotNull())).groupBy('Actress').count().sort((col('count')).desc())
western_actresses=data2.select('*').filter((data2.Genre == 'Western') & (data2.Actress.isNotNull())).groupBy('Actress').count().sort((col('count')).desc())
comedy_actresses=data2.select('*').filter((data2.Genre == 'Comedy') & (data2.Actress.isNotNull())).groupBy('Actress').count().sort((col('count')).desc())
action_actresses=data2.select('*').filter((data2.Genre == 'Action') & (data2.Actress.isNotNull())).groupBy('Actress').count().sort((col('count')).desc())
westerns_actresses=data2.select('*').filter((data2.Genre == 'Westerns') & (data2.Actress.isNotNull())).groupBy('Actress').count().sort((col('count')).desc())

In [44]:
dt0 = data2.filter(data2.Actress.isNotNull())
# Taking mode of Actress with most movies of a genre and adding it to the corresponding null values
actresses=data2.filter((data2.Genre == 'Drama') & (data2.Actress.isNotNull())).groupBy('Actress').count().sort((col('count')).desc())

maximum_val=actresses.agg({"count": "max"}).first()[0]

actress_with_most_movies=actresses.select('Actress').filter((col('count')==maximum_val)).collect()[0][0] # Mode
dt1 = data2.filter((data2.Genre == 'Drama')&(data2.Actress.isNull())).na.fill(value = actress_with_most_movies,subset=["Actress"])
actress_with_most_movies

'Bergman, Ingrid'

In [45]:
war_actresses=data2.select('*').filter((data2.Genre == 'War') & (data2.Actress.isNotNull())).groupBy('Actress').count().sort((col('count')).desc())
maximum_val=war_actresses.agg({"count": "max"}).first()[0]
actress_with_most_movies=war_actresses.select('Actress').filter((col('count')==maximum_val)).collect()[0][0] # Mode
dt2 = data2.filter((data2.Genre == 'War')&(data2.Actress.isNull())).na.fill(value = actress_with_most_movies,subset=["Actress"])
actress_with_most_movies

'Bow, Clara'

In [46]:
mystery_actresses=data2.select('*').filter((data2.Genre == 'Mystery') & (data2.Actress.isNotNull())).groupBy('Actress').count().sort((col('count')).desc())
maximum_val=mystery_actresses.agg({"count": "max"}).first()[0]

actress_with_most_movies=mystery_actresses.select('Actress').filter((col('count')==maximum_val)).collect()[0][0] # Mode

dt3= data2.filter((data2.Genre == 'Mystery')&(data2.Actress.isNull())).na.fill(value = actress_with_most_movies,subset=["Actress"])
actress_with_most_movies

'Ford, Maria'

In [47]:
music_actresses=data2.select('*').filter((data2.Genre == 'Music') & (data2.Actress.isNotNull())).groupBy('Actress').count().sort((col('count')).desc())
maximum_val=music_actresses.agg({"count": "max"}).first()[0]

actress_with_most_movies=music_actresses.select('Actress').filter((col('count')==maximum_val)).collect()[0][0] # Mode

dt4 = data2.filter((data2.Genre == 'Music')&(data2.Actress.isNull())).na.fill(value = actress_with_most_movies,subset=["Actress"])
actress_with_most_movies

'Madonna'

In [48]:
sci_fi_actresses=data2.select('*').filter((data2.Genre == 'Science Fiction') & (data2.Actress.isNotNull())).groupBy('Actress').count().sort((col('count')).desc())
maximum_val=sci_fi_actresses.agg({"count": "max"}).first()[0]

actress_with_most_movies=sci_fi_actresses.select('Actress').filter((col('count')==maximum_val)).collect()[0][0] # Mode

dt5 = data2.filter((data2.Genre == 'Science Fiction')&(data2.Actress.isNull())).na.fill(value = actress_with_most_movies,subset=["Actress"])
actress_with_most_movies

'Weaver, Sigourney'

In [49]:
horror_actresses=data2.select('*').filter((data2.Genre == 'Horror') & (data2.Actress.isNotNull())).groupBy('Actress').count().sort((col('count')).desc())
maximum_val=horror_actresses.agg({"count": "max"}).first()[0]
actress_with_most_movies=horror_actresses.select('Actress').filter((col('count')==maximum_val)).collect()[0][0] # Mode
dt6 = data2.filter((data2.Genre == 'Horror')&(data2.Actress.isNull())).na.fill(value = actress_with_most_movies,subset=["Actress"])
actress_with_most_movies

'Carlisle, Mary'

In [50]:
western_actresses=data2.select('*').filter((data2.Genre == 'Western') & (data2.Actress.isNotNull())).groupBy('Actress').count().sort((col('count')).desc())
maximum_val=western_actresses.agg({"count": "max"}).first()[0]

actress_with_most_movies=western_actresses.select('Actress').filter((col('count')==maximum_val)).collect()[0][0] # Mode

dt7 = data2.filter((data2.Genre == 'Western')&(data2.Actress.isNull())).na.fill(value = actress_with_most_movies,subset=["Actress"])
actress_with_most_movies

"O'Day, Nell"

In [51]:
comedy_actresses=data2.select('*').filter((data2.Genre == 'Comedy') & (data2.Actress.isNotNull())).groupBy('Actress').count().sort((col('count')).desc())
maximum_val=comedy_actresses.agg({"count": "max"}).first()[0]

actress_with_most_movies=comedy_actresses.select('Actress').filter((col('count')==maximum_val)).collect()[0][0] # Mode

dt8 = data2.filter((data2.Genre == 'Comedy')&(data2.Actress.isNull())).na.fill(value = actress_with_most_movies,subset=["Actress"])
actress_with_most_movies

'Hawn, Goldie'

In [52]:
action_actresses=data2.select('*').filter((data2.Genre == 'Action') & (data2.Actress.isNotNull())).groupBy('Actress').count().sort((col('count')).desc())
maximum_val=action_actresses.agg({"count": "max"}).first()[0]

actress_with_most_movies=action_actresses.select('Actress').filter((col('count')==maximum_val)).collect()[0][0] # Mode

dt9 = data2.filter((data2.Genre == 'Action')&(data2.Actress.isNull())).na.fill(value = actress_with_most_movies,subset=["Actress"])
actress_with_most_movies

'Loren, Sophia'

In [53]:
westerns_actresses=data2.select('*').filter((data2.Genre == 'Westerns') & (data2.Actress.isNotNull())).groupBy('Actress').count().sort((col('count')).desc())
maximum_val=westerns_actresses.agg({"count": "max"}).first()[0]

actress_with_most_movies=westerns_actresses.select('Actress').filter((col('count')==maximum_val)).collect()[0][0] # Mode

dt10 = data2.filter((data2.Genre == 'Westerns')&(data2.Actress.isNull())).na.fill(value = actress_with_most_movies,subset=["Actress"])
actress_with_most_movies

'Bardot, Brigitte'

In [54]:
data3 = unionAll(dt0,dt1,dt2,dt3,dt4,dt5,dt6,dt7,dt8,dt9,dt10)
print((data3.count(), len(data3.columns)))

(1657, 10)


5. Director
#### Director has 253 missing values here I will check with the same technique as used for Actor and then see what can we do to impute the values first

In [55]:
# First step is to check the Director we need to calculate the mode of the movies Genre for
data3.filter(data3.Director.isNull()).show(5)
# Since we have Null values for the Director working in almost all genre so I am going to take the most popular director in genre and based on the popularity of the movie I will purge the values
data3.select('Genre','Popularity').filter(data3.Director.isNull()).distinct().orderBy((data3.Popularity).desc()).show(5)
data3.select('Genre').filter(data3.Director.isNull()).distinct().show()

+----+------+--------------------+------+---------------+---------------+--------+----------+------+-------------------+
|Year|Length|               Title| Genre|          Actor|        Actress|Director|Popularity|Awards|              Image|
+----+------+--------------------+------+---------------+---------------+--------+----------+------+-------------------+
|1992|    90|           Germicide| Drama|    Taylor, Rod|Andersson, Bibi|    null|        36|    No|NicholasCage.png,,,|
|1954|   103|       Elephant Walk| Drama|   Finch, Peter|  Andrews, Dana|    null|        11|    No|NicholasCage.png,,,|
|1985|    55|Gonzo Presents Mu...|Comedy|   Cleese, John| Andrews, Julie|    null|        88|    No|NicholasCage.png,,,|
|1991|   115|              Bataan|   War| Taylor, Robert|    Arnaz, Desi|    null|        68|    No|NicholasCage.png,,,|
|1991|    90|Henry, Portrait o...|Horror|Rooker, Michael|  Arnold, Tracy|    null|        69|    No| NicholasCage.png,,|
+----+------+-------------------

In [56]:
dt0 = data3.filter(data3.Director.isNotNull())
drama_dt = data3.select('Director','Popularity').filter((data3.Genre == 'Drama') & (data3.Genre.isNotNull())).collect()

most_popular_director= ''
m = 0

for row in drama_dt:
    if(row[1] != None):
        if(m<row[1]):
            most_popular_director = row[0]
            m = row[1]
            
dt1 = data3.filter((data3.Genre == 'Drama')&(data3.Director.isNull())).na.fill(value = most_popular_director,subset=["Director"])
drama_dt = data3.select('Director','Popularity').filter((data3.Genre == 'War') & (data3.Genre.isNotNull())).collect()

most_popular_director= ''
m = 0

for row in drama_dt:
    if(row[1] != None):
        if(m<row[1]):
            most_popular_director = row[0]
            m = row[1]
            
dt2 = data3.filter((data3.Genre == 'War')&(data3.Director.isNull())).na.fill(value = most_popular_director,subset=["Director"])
drama_dt = data3.select('Director','Popularity').filter((data3.Genre == 'Mystery') & (data3.Genre.isNotNull())).collect()

most_popular_director= ''
m = 0

for row in drama_dt:
    if(row[1] != None):
        if(m<row[1]):
            most_popular_director = row[0]
            m = row[1]
            
dt3 = data3.filter((data3.Genre == 'Mystery')&(data3.Director.isNull())).na.fill(value = most_popular_director,subset=["Director"])
drama_dt = data3.select('Director','Popularity').filter((data3.Genre == 'Music') & (data3.Genre.isNotNull())).collect()

most_popular_director= ''
m = 0

for row in drama_dt:
    if(row[1] != None):
        if(m<row[1]):
            most_popular_director = row[0]
            m = row[1]
            
dt4 = data3.filter((data3.Genre == 'Music')&(data3.Director.isNull())).na.fill(value = most_popular_director,subset=["Director"])

drama_dt = data3.select('Director','Popularity').filter((data3.Genre == 'Science Fiction') & (data3.Genre.isNotNull())).collect()

most_popular_director= ''
m = 0

for row in drama_dt:
    if(row[1] != None):
        if(m<row[1]):
            most_popular_director = row[0]
            m = row[1]
            
dt5 = data3.filter((data3.Genre == 'Science Fiction')&(data3.Director.isNull())).na.fill(value = most_popular_director,subset=["Director"])
drama_dt = data3.select('Director','Popularity').filter((data3.Genre == 'Western') & (data3.Genre.isNotNull())).collect()

most_popular_director= ''
m = 0

for row in drama_dt:
    if(row[1] != None):
        if(m<row[1]):
            most_popular_director = row[0]
            m = row[1]
            
dt6 = data3.filter((data3.Genre == 'Western')&(data3.Director.isNull())).na.fill(value = most_popular_director,subset=["Director"])
drama_dt = data3.select('Director','Popularity').filter((data3.Genre == 'Comedy') & (data3.Genre.isNotNull())).collect()
most_popular_director= ''
m = 0

for row in drama_dt:
    if(row[1] != None and row[0] !=None):
        if(m<row[1]):
            most_popular_director = row[0]
            m = row[1]
dt7 = data3.filter((data3.Genre == 'Comedy')&(data3.Director.isNull())).na.fill(value = most_popular_director,subset=["Director"])
drama_dt = data3.select('Director','Popularity').filter((data3.Genre == 'Action') & (data3.Genre.isNotNull())).collect()

most_popular_director= ''
m = 0

for row in drama_dt:
    if(row[1] != None):
        if(m<row[1]):
            most_popular_director = row[0]
            m = row[1]
            
dt8 = data3.filter((data3.Genre == 'Action')&(data3.Director.isNull())).na.fill(value = most_popular_director,subset=["Director"])
drama_dt = data3.select('Director','Popularity').filter((data3.Genre == 'Westerns') & (data3.Genre.isNotNull())).collect()

most_popular_director= ''
m = 0

for row in drama_dt:
    if(row[0]!=None and row[1] != None):
        if(m<row[1]):
            most_popular_director = row[0]
            m = row[1]
            
dt9 = data3.filter((data3.Genre == 'Westerns')&(data3.Director.isNull())).na.fill(value = most_popular_director,subset=["Director"])
drama_dt = data3.select('Director','Popularity').filter((data3.Genre == 'Horror') & (data3.Genre.isNotNull())).collect()

most_popular_director= ''
m = 0

for row in drama_dt:
    if(row[0]!=None and row[1] != None):
        if(m<row[1]):
            most_popular_director = row[0]
            m = row[1]
            
dt10 = data3.filter((data3.Genre == 'Horror')&(data3.Director.isNull())).na.fill(value = most_popular_director,subset=["Director"])

In [57]:
data4 = unionAll(dt0,dt1,dt2,dt3,dt4,dt5,dt6,dt7,dt8,dt9,dt10)
print((data4.count(), len(data4.columns)))

(1657, 10)


6. Popularity
#### Popularity has only 6 missing values so here I will purge the values with the mean of Popularity

In [58]:
from pyspark.sql.functions import mean as _mean, col
m=0
m=data.agg(_mean('Popularity')).collect()
mea = m[0]['avg(Popularity)']
print(mea)

43.11010284331518


In [59]:
data4 = data4.na.fill(value = mea,subset=["Popularity"])

# New data has no null values

In [60]:
from pyspark.sql.functions import isnull, when ,count,col
# Original Dataset
dataorg.select([count(when(isnull(x), x)).alias(x) for x in dataorg.columns]).show()
# Dataset After Pre Processing
data4.select([count(when(isnull(x), x)).alias(x) for x in data4.columns]).show()

+----+------+-----+-----+-----+-------+--------+----------+------+-----+
|Year|Length|Title|Genre|Actor|Actress|Director|Popularity|Awards|Image|
+----+------+-----+-----+-----+-------+--------+----------+------+-----+
|   0|    67|    0|    2|    8|    378|     253|         6|     0|    0|
+----+------+-----+-----+-----+-------+--------+----------+------+-----+

+----+------+-----+-----+-----+-------+--------+----------+------+-----+
|Year|Length|Title|Genre|Actor|Actress|Director|Popularity|Awards|Image|
+----+------+-----+-----+-----+-------+--------+----------+------+-----+
|   0|     0|    0|    0|    0|      0|       0|         0|     0|    0|
+----+------+-----+-----+-----+-------+--------+----------+------+-----+



In [61]:
data4.toPandas().to_csv('Pre_Processed_Data.csv')

# I am using too many partitions. Hence, I am going do the remaining parts on the Pre Processed dataset on a new Session and will read it in a separate python notebook